In [1]:
# this is our preamble cell :
import plotly.graph_objects as go
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt

import category_encoders as ce
from sklearn.model_selection import train_test_split


from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder

from sklearn import cluster
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree

# from shap import TreeExplainer, initjs, force_plot
from pdpbox.pdp import pdp_interact, pdp_interact_plot

In [2]:
# importing and minor cleaning first, parsing dates. 

dfreal = pd.read_csv('True.csv',
                    parse_dates = ['date'])
#                    index_col = 'date')
dfreal['Fake'] = 0
print('Real data shape: ', dfreal.shape)

dffake = pd.read_csv('Fake.csv',
                    parse_dates = ['date'])
#                    index_col = 'date')

dffake['Fake'] = 1

# I have added the boolean column for Fake/Real to each. 

# This next bit will clean up the messy date columns from the fake csv. 

searchfor = ['http', '-', 'MSNBC']
dffake = dffake[~dffake['date'].str.contains('|'.join(searchfor))]

print('Fake data shape: ', dffake.shape)

Real data shape:  (21417, 5)
Fake data shape:  (23436, 5)


In [6]:
# Now, I want to trim them to be the same size, so that we have our baseline right at 0.5 -the same as flipping a coin. 

dfreal_trimmed = dfreal[-21_400 :]


dffake_trimmed = dffake[-21_400 :]


# and now combine them into one dataframe:
df_joined = dfreal_trimmed.append(dffake_trimmed, ignore_index=True)

df_joined['date'] = pd.to_datetime(df_joined['date'])
df_joined.drop(['subject'], axis=1, inplace=True)
df_joined.drop(['date'], axis=1, inplace=True)

df_joined = df_joined.drop(['title'], axis=1)

print(df_joined.columns)

Index(['text', 'Fake'], dtype='object')


In [7]:
target_body = df_joined['Fake']
Xb = df_joined['text']
yb = target_body


Xb_train, Xb_test, yb_train, yb_test = train_test_split(Xb, yb, test_size=0.2, random_state=42)

# doing 20/80 split and 42. and then the same to split val set from train set.

Xb_train, Xb_val, yb_train, yb_val = train_test_split(Xb_train, yb_train, test_size=0.2, random_state=42) 

print(Xb_train.shape)
print(yb_train.shape)
print(Xb_val.shape)
print(yb_val.shape)
print(Xb_test.shape)
print(yb_test.shape)

(27392,)
(27392,)
(6848,)
(6848,)
(8560,)
(8560,)


In [9]:
model_dash_TFIDF_5_50 = Pipeline([
    ('vectorizer',TfidfVectorizer(stop_words = 'english', strip_accents ='ascii', max_features = 100, min_df= 0.05 , max_df= 0.5)),
    #('dim_red', TruncatedSVD(n_components=19, random_state=42)),
    ('encoder', OrdinalEncoder()),
    ('predictor', GradientBoostingClassifier(random_state=42))
])

model_dash_TFIDF_5_50.fit(Xb_train, yb_train);
print('train: ', model_dash_TFIDF_5_50.score(Xb_train, yb_train))
print('val: ', model_dash_TFIDF_5_50.score(Xb_val, yb_val))
print('test: ',model_dash_TFIDF_5_50.score(Xb_test, yb_test))


train:  0.9325350467289719
val:  0.9192464953271028
test:  0.9184579439252336
